In [1]:
import json
import os
import random
import numpy as np
from utils import smiles_utils
from utils import polysmiles
import hoomd
import mbuild as mb
from mbuild.formats.hoomd_simulation import create_hoomd_simulation
import foyer
from foyer import Forcefield
import py3Dmol
import ele
import warnings
warnings.filterwarnings('ignore')

In [2]:
class Simulation():
    def __init__(self,
                 system,
                 target_box,
                 r_cut = 1.2,
                 e_factor = 0.5,
                 tau = 1,
                 dt = 0.001,
                 auto_scale = True,
                 ref_units = None,
                 mode = "gpu",
                 gsd_write = 1e4,
                 log_write = 1e3
                 ):
        
        self.system = system # Parmed structure
        self.r_cut = r_cut
        self.e_factor = e_factor
        self.tau = tau
        self.dt = dt
        self.auto_scale = auto_scale
        self.ref_units = ref_units
        self.target_box = target_box
        self.mode = mode
        self.gsd_write = gsd_write
        self.log_write = log_write
        
        if ref_units and not auto_scale:
            self.ref_energy = ref_units['energy']
            self.ref_distance = ref_units['distance']
            self.ref_mass = ref_units['mass']           
        
        elif auto_scale and not ref_units:
            self.ref_energy = 1
            self.ref_distance = 1
            self.ref_mass = 1
    
    def quench(self, kT, n_steps, shrink_kT, shrink_n_steps):
        
        # Get hoomd stuff set
        create_hoomd_simulation(self.system, self.ref_distance,
                                self.ref_mass, self.ref_energy,
                                self.r_cut, self.auto_scale)
        _all = hoomd.group.all()
        hoomd.md.integrate.mode_standard(dt=self.dt)
        integrator = hoomd.md.integrate.nvt(group=_all, kT=shrink_kT, tau=self.tau) # shrink temp
        integrator.randomize_velocities(seed=42)
        
        # Run shrinking step
        hoomd.dump.gsd("trajectories/traj-shrink.gsd",
                       period=self.gsd_write, group=_all, phase=0, overwrite=True)
        x_variant = hoomd.variant.linear_interp([(0, self.system.box[0]),
                                                 (shrink_n_steps, self.target_box[0]*10)])
        y_variant = hoomd.variant.linear_interp([(0, self.system.box[1]),
                                                 (shrink_n_steps, self.target_box[1]*10)])
        z_variant = hoomd.variant.linear_interp([(0, self.system.box[2]),
                                                 (shrink_n_steps, self.target_box[2]*10)])
        box_resize = hoomd.update.box_resize(Lx = x_variant, Ly = y_variant, Lz = z_variant)
        hoomd.run_upto(1e6)
        hoomd.dump.gsd.disable()
        box_resize.disable()
        
        # Run primary simulation
        hoomd.dump.gsd("trajectories/sim_traj.gsd",
                       period=self.gsd_write, group=_all, phase=0, overwrite=True)
        hoomd.analyze.log("logs/sim_traj.log",
                          period=self.log_write, group=_all, header_prefix="#",
                          overwrite=True, phase=0)
        
        integrator.set_params(kT=kT)
        integrator.randomize_velocities(seed=42)
        hoomd.run(n_steps+shrink_n_steps)
        
    def anneal(self,
              kT_init,
              kT_final,
              step_sequence):
        pass

class System():
    def __init__(self,
                 molecule,
                 para_weight,
                 density,
                 n_compounds,
                 polymer_lengths,
                 forcefield=None,
                 pdi=None,
                 M_n=None,
                 remove_hydrogens=False
                ):
        self.molecule = molecule
        self.para_weight = para_weight
        self.density = density
        self.remove_hydrogens = remove_hydrogens
        self.pdi = pdi
        self.forcefield = forcefield
        self.system_mass = 0
        self.para = 0 # keep track for now to check things are working, maybe keep?
        self.meta = 0
        
        if self.pdi:
            pass
            '''
            Here, call a function that samples from some distribution
            pass in pdi, n_compounds, M_n?
            self.polymer_lengths and self.n_compounds defined from that function
            '''
        else: # Do some validation, get things in the correct data types
            if not isinstance(n_compounds, list):
                self.n_compounds = [n_compounds]
            else:
                self.n_compounds = n_compounds

            if not isinstance(polymer_lengths, list):
                self.polymer_lengths = [polymer_lengths]
            else:
                self.polymer_lengths = polymer_lengths
        
        if len(self.n_compounds) != len(self.polymer_lengths):
            raise ValueError('n_compounds and polymer_lengths should be equal length')
        
        self.system = self.pack() # mBuild object before applying FF
        if self.forcefield:
            self.system = self.type_system() # parmed object after applying FF
        
        
    def pack(self, box_expand_factor=5):
        mb_compounds = []
        for _length, _n in zip(self.polymer_lengths, self.n_compounds):
            for i in range(_n):
                polymer, sequence = build_molecule(self.molecule, _length,
                                        self.para_weight)

                mb_compounds.append(polymer)
                self.para += sequence.count('para')
                self.meta += sequence.count('meta')
            mass = _n * np.sum(ele.element_from_symbol(p.name).mass for p in polymer.particles())
            self.system_mass += mass
        
        # Figure out correct box dimensions and expand the box to make the PACKMOL step faster
        # Will shrink down to accurate L during simulation
        L = self._calculate_L() * box_expand_factor
    
        system = mb.packing.fill_box(
            compound = mb_compounds,
            n_compounds = [1 for i in mb_compounds],
            box=[L, L, L],
            edge=0.5,
            fix_orientation=True)
        return system
    
    
    def type_system(self):
        if self.forcefield == 'gaff':
            forcefield = foyer.forcefields.load_GAFF()
        elif self.forcefield == 'opls':
            forcefield = foyer.Forcefield(name='oplsaa')
        
        typed_system = forcefield.apply(self.system)
        if self.remove_hydrogens: # not sure how to do this with Parmed yet
            removed_hydrogen_count = 0 # subtract from self.mass
            pass    
        return typed_system
    
    def _calculate_L(self):
        # Conversion from (amu/(g/cm^3)) to ang
        L = (self.system_mass / self.density) ** (1/3) * 1.841763
        L /= 10 # convert ang to nm
        return L
        

def build_molecule(molecule, length, para_weight):
    '''
    `build_molecule` uses SMILES strings to build up a polymer from monomers.
    The configuration of each monomer is determined by para_weight and the
    random_sequence() function.  
    Uses DeepSMILES behind the scenes to build up SMILES string for a polymer.
    
    Parameters
    ----------
    molecule : str
        The monomer molecule to be used to build up the polymer.
        Available options are limited  to the .json files in the compounds directory
        Use the molecule name as seen in the .json file without including .json
    length : int
        The number of monomer units in the final polymer molecule
    para_weight : float, limited to values between 0 and 1
        The relative amount of para configurations compared to meta.
        Passed into random_sequence() to determine the monomer sequence of the polymer.
        A 70/30 para to meta system would require a para_weight = 0.70
    
    Returns
    -------
    molecule_string_smiles : str
        The complete SMILES string of the polymer molecule
    '''
    f = open('compounds/{}.json'.format(molecule))
    mol_dict = json.load(f)    
    f.close()
    monomer_sequence = random_sequence(para_weight, length)
    molecule_string = '{}'

    for idx, config in enumerate(monomer_sequence):
        if idx == 0: # append template, but not brackets
            monomer_string = mol_dict['{}_template'.format(config)]
            molecule_string = molecule_string.format(monomer_string)
            if len(monomer_sequence) == 1:
                molecule_string = molecule_string.replace('{}', '')
                continue

        elif idx == length - 1: # Don't use template for last iteration
            brackets = polysmiles.count_brackets(mol_dict['{}_deep_smiles'.format(config)])
            monomer_string = mol_dict['{}_deep_smiles'.format(config)]
            molecule_string = molecule_string.format(monomer_string, brackets)

        else: # Continue using template plus brackets
            brackets = polysmiles.count_brackets(mol_dict['{}_deep_smiles'.format(config)])
            monomer_string = mol_dict['{}_template'.format(config)]
            molecule_string = molecule_string.format(monomer_string, brackets)
    
    molecule_string_smiles = smiles_utils.convert_smiles(deep = molecule_string)
    compound = mb.load(molecule_string_smiles, smiles=True)
    return compound, monomer_sequence


def random_sequence(para_weight, length):
    '''
    random_sequence returns a list containing a random sequence of strings 'para' and 'meta'.
    This is used by build_molecule() to create a complete SMILES string of a molecule.
    
    Parameters:
    -----------
    para_weight : float, limited to values between 0 and 1
        The relative amount of para configurations compared to meta.
        Defined in build_molecule()
    length : int
        The number of elements in the random sequence.
        Defined in build_molecule()
    '''
    meta_weight = 1 - para_weight
    options = ['para', 'meta']
    probability = [para_weight, meta_weight]
    sequence = random.choices(options, weights=probability, k=length)
    return sequence

In [3]:
test_system = System(molecule="PEEK", para_weight=0.60,
                    density=1.30, n_compounds=[10], polymer_lengths=[4],
                    forcefield='gaff')
print('done')

done


In [4]:
create_hoomd_simulation(test_system.system, r_cut=1.2, auto_scale=True)

HOOMD-blue v2.9.2-15-g8c1194a45 CUDA (10.1) SINGLE SSE SSE2 SSE3 SSE4_1 SSE4_2 AVX AVX2 
Compiled: 10/07/2020
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, J Glaser, and S C Glotzer. "HOOMD-blue: A Python package for
  high-performance molecular dynamics and hard particle Monte Carlo
  simulations", Computational Materials Science 173 (2020) 109363
-----
HOOMD-blue is running on the following GPU(s):
 [0]   GeForce GTX 1660 Ti  24 SM_7.5 @ 1.83 GHz, 5941 MiB DRAM, DIS, MNG


*Warning*: pos 376:-22.0352783203 -20.0463237762 -28.0467014313
*Warning*: fractional pos :0.107140824199 0.142601191998 -3.47195600625e-05
*Warning*: lo: -28.0447540283 -28.0447540283 -28.0447540283
*Warning*: hi: 28.0447540283 28.0447540283 28.0447540283
*Warning*: Not all particles were found inside the given box


RuntimeError: Error initializing ParticleData

In [5]:
_all = hoomd.group.all()
hoomd.md.integrate.mode_standard(dt=0.0001)
integrator = hoomd.md.integrate.nvt(group=_all, kT=1, tau=1)
hoomd.dump.gsd('hoomd-test.gsd', period=500, group=_all, phase=0, overwrite=True)
integrator.randomize_velocities(seed=42)

In [ ]:
'''
_all = hoomd.group.all()
hoomd.md.integrate.mode_standard(dt=0.0001)
integrator = hoomd.md.integrate.nvt(group=_all, kT=1.0, tau=0.1)
hoomd.dump.gsd("trajectories/start-shrink.gsd", period=None, group=_all, overwrite=True)
hoomd.dump.gsd("trajectories/traj-shrink.gsd", period=1e5, group=_all, phase=0, overwrite=True)
integrator.randomize_velocities(seed=42);

hoomd.update.box_resize(L = hoomd.variant.linear_interp([(0,100), (1e6,20)], zero=0));
'''

In [6]:
x_variant = hoomd.variant.linear_interp([(0, test_system.system.box[0]),
                                         (1e5, test_system.system.box[0]/3)], zero=0)

y_variant = hoomd.variant.linear_interp([(0, test_system.system.box[1]),
                                         (1e5, test_system.system.box[1]/3)], zero=0)

z_variant = hoomd.variant.linear_interp([(0, test_system.system.box[2]),
                                         (1e5, test_system.system.box[2]/3)], zero=0)

hoomd.update.box_resize(Lx = x_variant, Ly = y_variant, Lz = z_variant)

In [7]:
#hoomd.update.box_resize(L = hoomd.variant.linear_interp([(0,test_system.system.box[0]), (1e5,100)], zero=0));

In [8]:
#hoomd.update.box_resize(L = hoomd.variant.linear_interp([(0,190.6857872), (1e5,150.0)], zero=0));

In [9]:
hoomd.run_upto(1e5)

** starting run **


**ERROR**: Particle with unique tag 1068 is no longer in the simulation box.

Cartesian coordinates: 
x: 147.211 y: -375.689 z: -122.747
Fractional coordinates: 
f.x: 1.27275 f.y: -1.47209 f.z: -0.144331
Local box lo: (-95.2514, -95.2514, -95.2514)
          hi: (95.2514, 95.2514, 95.2514)


RuntimeError: Error computing cell list

In [ ]:
simulation = Simulation(system=test_system.system,
                        target_box=[10,10,10])

In [ ]:
simulation.quench(kT=1.5, n_steps=1e6, shrink_kT=1, shrink_n_steps=1e4)

In [ ]:
'''
<Type name="opls_280" class="C_2" element="C" mass="12.01100" def="[C;X3]([O;X1])(C)C"
    desc="C in ketone" doi="10.1021/ja9621760"/>

<Type name="opls_145" class="CA" element="C" mass="12.01100" def="[C;X3;r6]1[C;X3;r6][C;X3;r6][C;X3;r6][C;X3;r6][C;X3;r6]1"
overrides="opls_141,opls_142" doi="10.1021/ja9621760"/>

Only save the atom types we need

'''



In [ ]:
##############################################

In [ ]:
def build_system(packing='bcc'):
    '''
    Generate a simple LJ particle system using hoomd's create_lattice function
    packing 
    '''
    hoomd.context.initialize("")
    if packing == 'fcc':
        system = hoomd.init.create_lattice(unitcell=(hoomd.lattice.fcc(a=1.58)), n=6)
    elif packing == 'bcc':
        system = hoomd.init.create_lattice(unitcell=(hoomd.lattice.bcc(a=1.29)), n=6)
    hoomd.dump.gsd('{}_system.gsd'.format(packing), group = hoomd.group.all(), period=None, overwrite=True)
    return system

def hoomd_simulation(system, temp, tau):  
    nl = hoomd.md.nlist.cell()
    lj = hoomd.md.pair.lj(r_cut=2.5, nlist=nl)
    lj.pair_coeff.set('A', 'A', alpha=1.0, epsilon=1.0, sigma=1.0)
    hoomd.md.integrate.mode_standard(dt=0.001)
    _all = hoomd.group.all()
    nvt = hoomd.md.integrate.nvt(group=_all, kT=temp, tau=tau)
    nvt.randomize_velocities(seed=23)
    hoomd.analyze.log(filename='{}-tau_out.log'.format(tau),
                      quantities=["time", "temperature", "potential_energy"],
                      period=100,
                      overwrite=True
                     )
    #hoomd.dump.gsd('tau-trajectory.gsd', period=5e3, group=_all, overwrite=True)
    hoomd.run(3e5)

# --------------------------------------------------------

PEEK:  
1 amu = 1.66054e-24 g  
1.32 g/cm^3  
1.32 g/nm^3  
C19H12O3  
monomer_amu = 288.302  

In [ ]:
def build_peek_system(num_mols, poly_length, density):
    '''
    This function uses mBuild's packing functionality to create a very low dense system of molecules
    to allow for easier packing. A short simulation is then ran using HoomD to shrink the system to the
    desired starting density.
    '''
    peek_poly_smi = polysmiles.poly_smiles('occccC=O)ccccOcc*c*ccc6)))))))cc6)))))))cc6',
                                      length=poly_length)
    peek_poly = mb.load(peek_poly_smi, smiles=True)
    
    mol_amu = poly_length * 288.302
    mol_grams = mol_amu * 1.66054e-24
    system_mass = mol_grams * num_mols
    edge_cm = (system_mass / density)**(1/3)
    edge_nm = edge_cm * 1e7

    init_box = mb.Box([edge_nm*5]*3)
    system = mb.fill_box(peek_poly, num_mols, init_box)
    
    GAFF = foyer.forcefields.load_GAFF()
    system_pmd = GAFF.apply(system)
    create_hoomd_simulation(system_pmd, r_cut=1.2, auto_scale=True)
    forces = [f for f in hoomd.context.current.forces 
                if not isinstance(f, hoomd.md.charge.pppm)]
    
    hoomd.context.current.forces = forces
    _all = hoomd.group.all()
    hoomd.md.integrate.mode_standard(dt=0.0001)
    integrator = hoomd.md.integrate.nvt(group=_all, kT=2.0, tau=0.1)
    hoomd.dump.gsd("trajectories/start-shrink.gsd", period=None, group=_all, overwrite=True)
    hoomd.dump.gsd("trajectories/traj-shrink.gsd", period=1e5, group=_all, phase=0, overwrite=True)
    integrator.randomize_velocities(seed=42);

    hoomd.update.box_resize(L = hoomd.variant.linear_interp([(0,edge_nm*50), (1e6,edge_nm*10)], zero=0))
    hoomd.run(1e6)
    gsd_restart = hoomd.dump.gsd("trajectories/out-shrink.gsd", period=None, group=_all, overwrite=True)
    gsd_restart.write_restart()

In [ ]:
build_peek_system(num_mols=20, poly_length=5, density=1.2)

# -------------------------------------------------------------------------

In [ ]:
# Smiles Strings
'''
DEEPSMILES:
PEEK-para = "occccoccccC=O)cccccc6)))))))cc6)))))))cc6"
PEEK-meta = "occcoccccC=O)cccccc6)))))))cc6)))))))ccc6"
PEKK-para = "ccccoccccC=o)ccccC=o))cc6)))))))cc6)))))))cc6"
PEKK-meta = "ccccoccccC=o)cccC=o))ccc6)))))))cc6)))))))cc6"

SMILES:
PEEK-para = "oc3ccc(oc2ccc(C(=O)c1ccccc1)cc2)cc3"
PEEK-meta = "oc3cc(oc2ccc(C(=O)c1ccccc1)cc2)ccc3"
PEKK-para = "c3ccc(oc2ccc(C(=o)c1ccc(C=o)cc1)cc2)cc3"
PEKK-meta = "c3ccc(oc2ccc(C(=o)c1cc(C=o)ccc1)cc2)cc3"
'''


# DeepSMILES with bonded atom indicated
'''
PEEK Para Bonded:
"occccoccccC=O)cc*c*ccc6)))))))cc6)))))))cc6"
"occccoccccC=O)ccc*c*cc6)))))))cc6)))))))cc6"
"occccoccccC=O)cccc*c*c6)))))))cc6)))))))cc6"

PEEK Meta Bonded:
"occcoccccC=O)ccc*c*cc6)))))))cc6)))))))ccc6"
"occcoccccC=O)cc*c*ccc6)))))))cc6)))))))ccc6"
"occcoccccC=O)cccc*c*c6)))))))cc6)))))))ccc6"

PEKK Para Bonded
"ccccoccccC=o)cccc*C*=o))cc6)))))))cc6)))))))cc6"
PEKK Meta Bonded
"ccccoccccC=o)ccc*C*=o))ccc6)))))))cc6)))))))cc6"
'''


# Build up dicts and save to json file
'''
PEKK_dict = {}

PEKK_dict['para_deep_smiles'] = "ccccoccccC=o)ccccC=o))cc6)))))))cc6)))))))cc6"
PEKK_dict['meta_deep_smiles'] = "ccccoccccC=o)cccC=o))ccc6)))))))cc6)))))))cc6"
PEKK_dict['para_smiles'] = "c3ccc(oc2ccc(C(=o)c1ccc(C=o)cc1)cc2)cc3"
PEKK_dict['meta_smiles'] = "c3ccc(oc2ccc(C(=o)c1cc(C=o)ccc1)cc2)cc3"
PEKK_dict['para_template'] = "ccccoccccC=o)cccc*C*=o))cc6)))))))cc6)))))))cc6"
PEKK_dict['meta_template'] = "ccccoccccC=o)ccc*C*=o))ccc6)))))))cc6)))))))cc6"
PEKK_dict['monomer_mass'] = 302.329


PEEK_dict = {}

PEEK_dict['para_deep_smiles'] = "occccoccccC=O)cccccc6)))))))cc6)))))))cc6"
PEEK_dict['meta_deep_smiles'] = "occcoccccC=O)cccccc6)))))))cc6)))))))ccc6"
PEEK_dict['para_smiles'] = "oc3ccc(oc2ccc(C(=O)c1ccccc1)cc2)cc3"
PEEK_dict['meta_smiles'] = "oc3cc(oc2ccc(C(=O)c1ccccc1)cc2)ccc3"
PEEK_dict['para_template'] = "occccoccccC=O)ccc{}{}ccc6)))))))cc6)))))))cc6"
PEEK_dict['meta_template'] = "occcoccccC=O)cccc{}{}cc6)))))))cc6)))))))ccc6"
PEEK_dict['monomer_mass'] = 288.302



file_name = 'compounds/PEKK.json'
with open(file_name, 'w') as fp:
    json.dump(PEKK_dict, fp)
'''